In [9]:
datasets = {
    "xor": {
        "train": "./dat/train_xor.dat",
        "test": "./dat/test_xor.dat"
    }
    # ,
    # "compas": {
    #     "train": "./dat/train_compas.dat",
    #     "test": "./dat/test_compas.dat"
    # },
    # "nomnist": {
    #     "train": "./dat/train_nomnist.dat",
    #     "test": "./dat/test_nomnist.dat"
    # }
}

In [10]:
import csv

def load_cma_results_and_get_pareto(csv_file):
    """
    Carga los resultados de CMA-ES desde un archivo CSV, calcula el conjunto de soluciones
    no dominadas (frente de Pareto) en base a error de entrenamiento, error de test,
    y número de neuronas, e imprime los no dominados.
    
    :param csv_file: Ruta del archivo CSV con los resultados de CMA-ES.
    :return: Una tupla (no_dominados, dominados), donde ambos son listas de individuos completos.
    """
    # Cargar resultados desde el archivo CSV
    results = []
    with open(csv_file, newline='') as csvfile:
        reader = csv.reader(csvfile)
        headers = next(reader)  # Leer encabezados
        for row in reader:
            results.append([float(x) if i < 6 else float(x) for i, x in enumerate(row)])
    
    # Extraer objetivos: train_error, test_error, total_neurons
    objectives = [
        (result[6], result[8], result[2] * result[3])  # Train error, Test error, Total neurons
        for result in results
    ]

    # Identificar no dominados y dominados
    no_dominados = []
    dominados = []

    for i, obj_i in enumerate(objectives):
        is_dominated = False
        for j, obj_j in enumerate(objectives):
            if i != j and dominates(obj_j, obj_i):  # Si obj_j domina a obj_i
                is_dominated = True
                break
        if is_dominated:
            dominados.append(results[i])
        else:
            no_dominados.append(results[i])

    # Imprimir no dominados
    print("No Dominados:")
    for nd in no_dominados:
        print(nd)

    return no_dominados, dominados


def dominates(ind1, ind2):
    """
    Determina si el individuo ind1 domina al individuo ind2.
    Un individuo domina a otro si es igual o mejor en todos los objetivos
    y estrictamente mejor en al menos uno.
    :param ind1: Primer individuo (objetivos).
    :param ind2: Segundo individuo (objetivos).
    :return: True si ind1 domina a ind2, False en caso contrario.
    """
    return all(x <= y for x, y in zip(ind1, ind2)) and any(x < y for x, y in zip(ind1, ind2))

In [11]:
import matplotlib.pyplot as plt
import os

def obtain_best_combination(pareto_results, dataset_key):
    """
    Analiza las combinaciones no dominadas y encuentra la mejor combinación basada en
    la relación beneficio-esfuerzo. Genera y guarda un gráfico para visualizar los resultados.
    
    :param pareto_results: Lista de resultados no dominados.
    :param dataset_key: Nombre del dataset (usado para nombrar el archivo de la imagen).
    :return: Índice de la mejor combinación y su métrica.
    """
    # Crear directorio si no existe
    output_dir = "resultados/imagenes"
    os.makedirs(output_dir, exist_ok=True)
    
    # Ordenar por complejidad
    pareto_results = sorted(pareto_results, key=lambda x: x[2] * x[3])  # Ordenar por total_neurons

    best_index = None
    best_efficiency = -float("inf")
    efficiencies = []

    # Graficar relación beneficio-esfuerzo
    plt.figure(figsize=(10, 6))
    for i in range(1, len(pareto_results)):
        prev = pareto_results[i - 1]
        curr = pareto_results[i]

        # Calcular incrementos
        prev_neurons = prev[2] * prev[3]
        curr_neurons = curr[2] * curr[3]
        complexity_increase = curr_neurons - prev_neurons

        train_gain = prev[6] - curr[6]  # Mejora en error de entrenamiento
        test_gain = prev[8] - curr[8]  # Mejora en error de prueba

        # Relación beneficio-esfuerzo
        total_gain = train_gain + test_gain
        if complexity_increase > 0:
            efficiency = total_gain / complexity_increase
        else:
            efficiency = 0  # Evitar divisiones por cero

        efficiencies.append(efficiency)

        # Actualizar la mejor combinación
        if efficiency > best_efficiency:
            best_efficiency = efficiency
            best_index = i

        # Añadir al gráfico
        plt.scatter(complexity_increase, total_gain, label=f"Comb {i}", s=80)

    # Graficar
    plt.xlabel("Complexity Growth (Total Neurons)")
    plt.ylabel("Total Gain (Train + Test Error)")
    plt.title(f"Benefit-Effort Ratio for Non-Dominated Combinations - {dataset_key}")
    plt.axhline(0, color='gray', linestyle='--')
    plt.legend()
    plt.grid(True)
    
    # Guardar la imagen
    output_file = os.path.join(output_dir, f"non_dominated_analysis_{dataset_key}.png")
    plt.savefig(output_file)
    plt.close()

    print(f"Gráfico guardado en: {output_file}")

    # Retornar la mejor combinación
    return best_index, pareto_results[best_index], best_efficiency


In [12]:
import re
from collections import defaultdict
import os
import subprocess
def leer_vectores(filepath):
    """
    Lee un archivo de salida y organiza los resultados por semilla.
    
    :param filepath: Ruta al archivo de salida.
    :return: Diccionario que contiene los datos organizados por semilla.
    """
    # Patrones para extraer la semilla y los datos de iteración/errores
    seed_pattern = re.compile(r"SEED (\d+)")
    iteration_pattern = re.compile(r"Iteration (\d+)\s+Training error:\s+([\d.]+)\s+Test error:\s+([\d.]+)")
    
    # Diccionario para almacenar resultados
    results = defaultdict(lambda: {"iterations": [], "training_errors": [], "test_errors": []})
    
    # Variables auxiliares
    current_seed = None

    # Leer el archivo línea por línea
    with open(filepath, "r") as file:
        for line in file:
            # Identificar la semilla actual
            seed_match = seed_pattern.search(line)
            if seed_match:
                current_seed = int(seed_match.group(1))
                continue
            
            # Extraer los datos de iteración/errores si hay una semilla activa
            if current_seed is not None:
                iteration_match = iteration_pattern.search(line)
                if iteration_match:
                    iteration = int(iteration_match.group(1))
                    training_error = float(iteration_match.group(2))
                    test_error = float(iteration_match.group(3))
                    
                    # Agregar los datos al diccionario de la semilla actual
                    results[current_seed]["iterations"].append(iteration)
                    results[current_seed]["training_errors"].append(training_error)
                    results[current_seed]["test_errors"].append(test_error)
    
    # Convertir el diccionario en un formato estándar
    return dict(results)

def execute_best_combinations(dataset_key, combination, output_filename):
    """
    Ejecuta una combinación de parámetros y guarda la salida en un archivo de texto.
    
    :param dataset_key: Clave del dataset.
    :param combination: Lista con los parámetros a ejecutar.
    :param output_filename: Nombre del archivo de salida.
    :return: Salida de la ejecución como texto.
    """
    train_file = datasets[dataset_key]["train"]
    test_file = datasets[dataset_key]["test"]
    layers, neurons, eta, mu, offline, error_function = combination[:6]

    command = [
        "./bin/la2",
        "-l", str(layers),
        "-h", str(neurons),
        "-e", str(eta),
        "-m", str(mu),
        "-o", str(offline),
        "-f", str(error_function),
        "-t", train_file,
        "-T", test_file,
        "-i", "1000"
    ]

    # Ejecutar el comando
    result = subprocess.run(command, capture_output=True, text=True)

    # Guardar la salida en un archivo
    output_dir = "resultados2/outputs"
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, output_filename)
    with open(output_path, "w") as file:
        file.write(result.stdout)

    return result.stdout



def create_convergence_plot(data, title, output_filename, max_iterations=1000):
    """
    Crea un gráfico de convergencia basado en los datos de la semilla número 5.
    
    :param data: Diccionario con los datos organizados por semillas.
    :param title: Título del gráfico.
    :param output_filename: Nombre del archivo donde se guardará el gráfico.
    :param max_iterations: Número máximo de iteraciones a considerar.
    """
    # Verificar si la semilla número 5 está presente
    if 5 not in data:
        print("La semilla número 5 no está disponible en los datos.")
        return
    
    # Extraer los datos de la semilla número 5
    seed_5_data = data[5]
    iterations = seed_5_data["iterations"]
    training_errors = seed_5_data["training_errors"]
    test_errors = seed_5_data["test_errors"]
    
    # Validar si el penúltimo valor no corresponde a la última iteración menos uno
    if len(iterations) > 1 and iterations[-2] != iterations[-1] - 1:
        print("Advertencia: Las iteraciones no son consecutivas al final. Se usarán datos hasta el penúltimo valor.")
        iterations = iterations[:-1]
        training_errors = training_errors[:-1]
        test_errors = test_errors[:-1]
    
    # Crear el gráfico
    plt.figure(figsize=(10, 6))
    plt.plot(iterations, training_errors, label="Training Error", color="orange")
    plt.plot(iterations, test_errors, label="Test Error", color="blue")
    plt.xlabel("Iterations")
    plt.ylabel("Error")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    
    # Guardar el gráfico
    plt.savefig(output_filename)
    plt.close()
    print(f"Gráfico de convergencia guardado en: {output_filename}")


def run_full_study():
    """
    Ejecuta el estudio completo según los pasos solicitados.
    """
    report_lines = []

    for dataset_key in datasets.keys():
        # 1. Cargar resultados de CMA
        csv_file = f"resultados2/grid_search_{dataset_key}.csv"
        with open(csv_file, newline='') as csvfile:
            reader = csv.reader(csvfile)
            headers = next(reader)
            results = [list(map(float, row)) for row in reader]

        # 2. Obtener el mejor resultado (menor train y test error)
        best_result = min(results, key=lambda x: (x[6], x[8]))

        # 3. Obtener soluciones no dominadas y la mejor combinación
        non_dominated, _ = load_cma_results_and_get_pareto(csv_file)
        _, best_pareto, _ = obtain_best_combination(non_dominated, dataset_key)

        # 4. Ejecutar el mejor resultado y el mejor no dominado
        print(f"Ejecutando la mejor combinación posible para {dataset_key}...")
        best_output = execute_best_combinations(dataset_key, best_result, f"mejor_posible_{dataset_key}_output.txt")
        print(f"Ejecutando la mejor combinación no dominada para {dataset_key}...")
        pareto_output = execute_best_combinations(dataset_key, best_pareto, f"mejor_no_dominado_{dataset_key}_output.txt")

        # 5. Obtener vectores y crear gráficos de convergencia
        print("Generando gráficos de convergencia...")
        best_data = leer_vectores(f"resultados2/outputs/mejor_posible_{dataset_key}_output.txt")
        pareto_data = leer_vectores(f"resultados2/outputs/mejor_no_dominado_{dataset_key}_output.txt")

        create_convergence_plot(
            best_data, "Error Convergence - Best Result", f"resultados2/imagenes/error_convergence_best_{dataset_key}.png"
        )
        create_convergence_plot(
            pareto_data, "Error Convergence - Optimal Pareto", f"resultados2/imagenes/error_convergence_optimal_{dataset_key}.png"
        )

        # 6. Agregar al informe
        report_lines.append(f"Dataset: {dataset_key}")
        report_lines.append(f"Mejor resultado posible: {best_result}")
        report_lines.append(f"Mejor combinación no dominada: {best_pareto}")
        report_lines.append("")

    # Guardar el informe
    with open("resultados2/outputs/informe.txt", "w") as report_file:
        report_file.write("\n".join(report_lines))
    print("Informe generado en: resultados2/outputs/informe.txt")


# Ejecutar el estudio completo
run_full_study()

No Dominados:
[0.01, 0.9, 4.0, 64.0, 0.0, 1.0, 3.79757e-07, 6.85018e-07, 3.79757e-07, 6.85018e-07]
[0.7, 0.9, 1.0, 2.0, 1.0, 1.0, 0.000125856, 1.62238e-05, 0.000125856, 1.62238e-05]
[0.7, 0.9, 1.0, 4.0, 1.0, 1.0, 8.74007e-05, 9.22675e-06, 8.74007e-05, 9.22675e-06]
[0.7, 0.9, 1.0, 8.0, 0.0, 1.0, 5.97174e-05, 2.58856e-06, 5.97174e-05, 2.58856e-06]
[0.7, 0.9, 1.0, 16.0, 0.0, 1.0, 3.17837e-05, 6.04986e-06, 3.17837e-05, 6.04986e-06]
[0.7, 0.9, 1.0, 32.0, 0.0, 1.0, 1.25663e-05, 1.11228e-05, 1.25663e-05, 1.11228e-05]
[0.7, 0.9, 2.0, 32.0, 0.0, 1.0, 4.4179e-06, 3.70672e-06, 4.4179e-06, 3.70672e-06]
[0.7, 0.9, 2.0, 64.0, 0.0, 1.0, 1.68057e-06, 3.30783e-06, 1.68057e-06, 3.30783e-06]
[0.7, 0.9, 3.0, 64.0, 0.0, 1.0, 9.07701e-07, 1.8154e-06, 9.07701e-07, 1.8154e-06]
Gráfico guardado en: resultados/imagenes/non_dominated_analysis_xor.png
Ejecutando la mejor combinación posible para xor...
Ejecutando la mejor combinación no dominada para xor...
Generando gráficos de convergencia...
Gráfico de converg